### **下載資料**
下載food-11資料集，包含11個類別的食物圖集

In [ ]:
# 下載資料
# !gdown --id '1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy' --output food-11.zip

# 解壓縮
# !unzip -q food-11.zip

### **安裝相關套件**
在這次作業中會需要用到torchvision這個PyTorch的library

In [1]:
# 安裝需要用到的套件
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image

# 在做semi-supervised learning時，"ConcatDataset"和"Subset"會有用處
from torch.utils.data import ConcatDataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder

# 用來做進度條
from tqdm.auto import tqdm

### **Datase，DataLoader and Transforms**
Torchvision提供許多有用的utilities，用於image preprocessing, data wrapping以及data augmentation

因為資料是按照標籤類別存在各個資料夾裡，因此可以直接用**torchvision.datasets.DatasetFolder**來包裝資料

可以參考下面的PyTorch官網來得知不同Transforms的細節

網址:https://pytorch.org/vision/stable/transforms.html

In [2]:
# 在訓練時做data augmentation很重要，但不是每個方法都有用
# 要根據任務來選擇方法
# train_tfm = transforms.Compose([
#     transforms.Resize((128, 128)),           # 調整圖像至固定大小(128x128)
#     transforms.RandomResizedCrop(128),       # 隨機裁剪部分影像並調整圖像大小
#     transforms.RandomHorizontalFlip(0.5),    # 以0.5的機率水平翻轉圖像
#     transforms.RandomRotation(90),           # 隨機旋轉圖像，範圍在-90度至90度
    # transforms.RandomPerspective(p=0.5),     # 以p的機率進行透視變換
#     transforms.ToTensor(),                   # 轉成Tensor(放在最後)
    # 對影像做normalization，可加快收斂速度
    # 下面的mean跟std是Imagenet給出的數據，如果想要更好的效果就要參考以下網址自己計算資料集的mean跟std
    # 參考網址:https://zhuanlan.zhihu.com/p/414242338
    # 參考網址:https://zj-image-processing.readthedocs.io/zh-cn/latest/pytorch/preprocessing/[%E6%95%B0%E6%8D%AE%E5%BD%92%E4%B8%80%E5%8C%96]%E5%9D%87%E5%80%BC%E5%92%8C%E6%96%B9%E5%B7%AE%E8%AE%BE%E7%BD%AE/
    # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
# ])

train_tfm1 = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
])
train_tfm2 = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.RandomResizedCrop(128),
        transforms.ToTensor(),
])
train_tfm3 = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.RandomHorizontalFlip(p=1),
        transforms.ToTensor(),
])
train_tfm4 = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.RandomRotation(90),
        transforms.ToTensor(),
])
train_tfm5 = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.RandomPerspective(p=1),
        transforms.ToTensor(),
])


# testingi跟validation不需要做augmentation
# 只需要重設大小並轉成Tensor
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

In [3]:
# 較大的batch size通常可以得到較穩定的gradient
# 但GPU memory有限制所以要小心調整
batch_size = 64

# 建構Datasets
# loader是決定讀取資料的方法(預設為cv2.imread)
# extensions是允許讀取的資料副檔名
train_set1 = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm1)
train_set2 = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm2) 
train_set3 = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm3)
train_set4 = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm4)
train_set5 = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm5)

train_set = ConcatDataset([train_set1, train_set2, train_set3, train_set4, train_set5])
valid_set = DatasetFolder("food-11/validation", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
unlabeled_set = DatasetFolder("food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm1)
test_set = DatasetFolder("food-11/testing", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)

# 建構DataLoaders(記得將num_workers=0，否則記憶體會吃不消，但計算速度會下降)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

### **Model**
由於彩色影像有RGB三個通道，因此網路的輸入通道為3個(灰階圖為1個)。在每個捲積層中，通道數通常會增加，而高度與寬度會減少(根據stride、padding或kernel size決定減少量，也有可能保持不變)

在輸入進fully-connected layers前，要先將從捲積層得到的feature map展開成單一一維向量，再經過fully-connected layers得到每個class的"logits"

"logits"通常是指softmax或sigmoid前的tensor，也就是fully-connected layers的output

注意:可以使用預訓練模型的架構(例如VGG、ResNet)，但不能載入預訓練的權重

In [4]:
# 實現Residual Block
class ResBlock(nn.Module):
    def __init__(self, inputchannel, outputchannel, stride=1):
        super(ResBlock, self).__init__()

        # Residual Function
        self.resfunc = nn.Sequential(
            nn.Conv2d(inputchannel, outputchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outputchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outputchannel, outputchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outputchannel)
        )

        # Shortcut
        self.shortcut = nn.Sequential()
        # 要確保跟Residual Function輸出的大小、通道數一致而對input進行處理
        if stride != 1 or outputchannel != inputchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inputchannel, outputchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outputchannel)
            )

    def forward(self, x):
        out = self.resfunc(x) + self.shortcut(x)
        out = nn.ReLU(inplace=True)(out)

        return out     

In [6]:
class ResNet(nn.Module):
    def __init__(self, ResBlock):
        super(ResNet, self).__init__()

        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self.make_layer(ResBlock, 64, 2, stride=1)
        self.layer2 = self.make_layer(ResBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResBlock, 512, 2, stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, 11)

    def make_layer(self, block, outchannel, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)              #兩個list相加([stride, 1*(num_blocks - 1)])
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, outchannel, stride))
            self.inchannel = outchannel
        return nn.Sequential(*layers)                            # 用*指向layers裡的元素
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.max_pool(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)

        # 輸入圖片大小為3*128*128(C*H*W)
        # 計算卷積或池化層的output大小可以用以下公式
        # output_size = (input_size + 2*padding - kernel_size) / 2 + 1
        # 捲積層
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 32, 5, 2, 2),     # [32, 64, 64]
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),         # [32, 32, 32]
            
            nn.Conv2d(32, 64, 3, 1, 1),    # [64, 32, 32]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),         # [64, 16, 16]

            nn.Conv2d(64, 128, 3, 1, 1),   # [128, 16, 16]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            # nn.MaxPool2d(2, 2, 0),         # [128, 16, 16]

            nn.Conv2d(128, 256, 3, 1, 1),  # [256, 16, 16]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),         # [256, 8, 8]
        )
        # 全連接層
        # 第1個Linear layer的input dim大小等於捲積層output的C*H*W(H*W根據stride, padding自行計算)
        self.fc_layers = nn.Sequential(
            nn.Linear(256 * 8 * 8, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.Linear(256, 11),
        )

    def forward(self, x):
        # 輸入(x): [batch_size, 3, 128, 128]
        # 輸出: [batch_size, 11]

        # 用捲積層提取圖像特徵
        x = self.cnn_layers(x)

        # 將從捲積層得到的feature map展開成單一一維向量.
        x = x.flatten(1)

        # 透過全連接層計算得到每個class的"logits"
        x = self.fc_layers(x)
        return x

### Training
執行sample code就能完成supervised learning，但使用未標註的資料進行semi-supervised learning可以得到更好的效果，下面的get_pseudo_labels函式就是用於semi-supervised learning，需要自己完成

關於semi-supervised learning可以到以下網址觀看老師的課程

網址:https://www.youtube.com/watch?v=fX_guE7JNnY

In [ ]:
# get_pseudo_labels會用給定的model產生資料集的pseudo-labels
# 它會回傳一個DatasetFolder的實例，包含預測置信度超過閾值的圖象
def get_pseudo_labels(dataset, model, threshold=0.65):
    # 判斷裝置
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # 建構DataLoader
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    model.eval()          #將model設為evalutation mode
    # 定義softmax函式(dim有0、1、2、-1，其中-1與2效果一樣，是對某一維度的每一行做softmax)
    # dim=1是對某一維度的每一列做softmax
    # dim=0是對每個維度的相同位置做softmax
    softmax = nn.Softmax(dim=-1)

    # 以batch為單位對資料集進行迭代並顯示進度條
    for batch in tqdm(data_loader):
        img, _ = batch

        # 關掉梯度計算來加速model的forward
        with torch.no_grad():
            logits = model(img.to(device))

        # 將logits進行softmax後得到機率分布
        probs = softmax(logits)

        # ---------- TODO ----------
        # Filter the data and construct a new dataset.
        

    ## 將model設為train
    model.train()
    return dataset

In [7]:
def same_seeds(seed):
    torch.manual_seed(seed)     # 為CPU設定random seed
    if torch.cuda.is_available():
        # torch.cuda.manual_seed(seed)    # 為特定GPU設定random seed
        torch.cuda.manual_seed_all(seed)  # 為所有GPU設定random seed
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False    # 最佳化卷積層
    torch.backends.cudnn.deterministic = True   # 讓GPU的輸出一致

In [8]:
# 判斷裝置
device = "cuda" if torch.cuda.is_available() else "cpu"
same_seeds(0)

# 初始化model並放到device上
# model = Classifier().to(device)
model = ResNet(ResBlock=ResBlock).to(device)
model.device = device

model_path = './model.ckpt'

# 定義loss function為cross entropy
criterion = nn.CrossEntropyLoss()

# 初始化optimizer並調整hyperparameters
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)

# 設定epoch次數
n_epochs = 100

# 決定是否要做semi-supervised learning
do_semi = False

best_acc = 0.0

for epoch in range(n_epochs):
    # ---------- TODO ----------
    # In each epoch, relabel the unlabeled dataset for semi-supervised learning.
    # Then you can combine the labeled dataset and pseudo-labeled dataset for the training.
    if do_semi:
        # Obtain pseudo-labels for unlabeled data using trained model.
        pseudo_set = get_pseudo_labels(unlabeled_set, model)

        # Construct a new dataset and a data loader for training.
        # This is used in semi-supervised learning only.
        concat_dataset = ConcatDataset([train_set, pseudo_set])
        train_loader = DataLoader(concat_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

    # ---------- Training ----------
    model.train()   # 設置model為train mode

    # 紀錄training loss跟training accuracy
    train_loss = []
    train_accs = []

    # 以batch為單位對資料集進行迭代並顯示進度條
    for batch in tqdm(train_loader):
        imgs, labels = batch    # 每個batch包含圖像跟對應的label
        logits = model(imgs.to(device)) # Forward the data. (確保資料跟model在同個device上)

        # 計算cross-entropy loss.
        # PyTorch在計算cross-entropy loss時會自動進行softmax，所以不用自己做softmax
        loss = criterion(logits, labels.to(device))
        optimizer.zero_grad()    # 清除前面留下的gradient
        loss.backward()       # 計算網路參數的gradient

        # 網路較大時，梯度計算可能會因連乘項變多而發生梯度爆炸或消失
        # 透過nn.utils.clip_grad_norm_ function限制gradient norm來避免梯度爆炸的問題
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
        optimizer.step()       # 更新網路參數
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()   # 計算train accuracy

        # 紀錄loss跟accuracy(training)
        train_loss.append(loss.item())
        train_accs.append(acc)

    # 計算average loss跟accuracy
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    model.eval()      # 設置model為evalutation mode

    # 紀錄validation loss跟validation accuracy
    valid_loss = []
    valid_accs = []

    # 以batch為單位對資料集進行迭代並顯示進度條
    for batch in tqdm(valid_loader):
        imgs, labels = batch    # 每個batch包含圖像跟對應的label

        # 關掉梯度計算來加速model的forward
        with torch.no_grad():
          logits = model(imgs.to(device))
        loss = criterion(logits, labels.to(device))   # 計算cross-entropy loss
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean() # 計算accuracy

        # 紀錄loss跟accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)

    # 計算average loss跟accuracy(validation)
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    if valid_acc > best_acc:
        best_acc = valid_acc
        torch.save(model.state_dict(), model_path)
        print('Save the model')

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

  0%|          | 0/241 [00:00<?, ?it/s]

[ Train | 001/100 ] loss = 1.93947, acc = 0.32151


  0%|          | 0/11 [00:00<?, ?it/s]

Save the model
[ Valid | 001/100 ] loss = 1.73607, acc = 0.38580


  0%|          | 0/241 [00:00<?, ?it/s]

...

[ Valid | 099/100 ] loss = 1.40200, acc = 0.73977


  0%|          | 0/241 [00:00<?, ?it/s]

[ Train | 100/100 ] loss = 0.07981, acc = 0.97316


  0%|          | 0/11 [00:00<?, ?it/s]

[ Valid | 100/100 ] loss = 1.79199, acc = 0.71932


### **Testing**


In [10]:
# model = Classifier().to(device)
model = ResNet(ResBlock=ResBlock).to(device)

model.load_state_dict(torch.load(model_path))

model.eval()  # 設置model為evalutation mode
predictions = []  # 紀錄預測結果

for batch in tqdm(test_loader):
    # 這裡的labels全部為0，沒有任何意義
    # 因為DatasetFolder要回傳每個batch的image跟label
    # 所以要創造假的label來確保其能正常運作
    imgs, labels = batch

    # 關掉梯度計算來加速model的forward
    with torch.no_grad():
        logits = model(imgs.to(device))

    # 紀錄機率最大的位置做為預測結果
    # extend:在list後面增加一整個list的元素
    predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

  0%|          | 0/53 [00:00<?, ?it/s]

In [11]:
# 將預測結果儲存至檔案
with open("predict.csv", "w") as f:
    f.write("Id,Category\n")    # 第一列為"Id, Category"

    # 其餘列為圖像ID跟其class的預測結果
    for i, pred in  enumerate(predictions):
         f.write(f"{i},{pred}\n")